# Web Scraping from Indeed Website

* Log: 02082017 Jiewei Chen Da Xue
* Log: 02102016 Jiewei Chen Da Xue Nick Ulle
* Log: 02212017 Jiewei Chen

## Step 1. Using API get 5000 Posts

In [47]:
import requests
import requests_cache
requests_cache.install_cache("cache")
import pandas as pd
from bs4 import BeautifulSoup


import string
import nltk
#from nltk import corpus
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer   # sklearn --- primer machine learning package
from sklearn.neighbors import NearestNeighbors

In [2]:
publisher_id = "254802918413674"

http://api.indeed.com/ads/apisearch?publisher=254802918413674&q=java&l=austin%2C+tx&sort=&radius=&st=&jt=&start=&limit=&fromage=&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2

In [50]:
url = "http://api.indeed.com/ads/apisearch?publisher=254802918413674&format=json&q=data&l=sunnyvale%2C+ca&sort=&radius=0&st=&jt=internship&start=&limit=1000&fromage=30&filter=&latlong=1&co=us&chnl=&userip=1.2.3.4&useragent=Mozilla/%2F4.0%28Firefox%29&v=2"

In [3]:
def get_indeed(city, state, keyword, radius, jobtype = "internship", js = "json", start = 0, publisher_id = publisher_id):
    url = "http://api.indeed.com/ads/apisearch?"
    if city == "":
        location = state
    else: 
        location = city + ', ' + state
    result = requests.get(url, params = {
            "q": keyword,
            "v": 2,
            "format": js,
            "publisher": publisher_id,
            "l": location,
            "jt": jobtype, # "fulltime", "parttime", "contract", "internship", "temporary"
            "radius": radius,
            "start": start,
            "limit": 100})
    result = result.json()['results']
    return result

In [4]:
job_list = pd.DataFrame()
for i in range(0,5001,25):
    temp = pd.DataFrame(get_indeed('','ca', '"data analyst"', 0, start = i))
    job_list = job_list.append(temp)


Save Result to a csv file, so that next time we run this code it will be faster. Don't need to reload from website.

In [88]:
# job_list.to_csv('job_ca_dataanalyst.csv')

In [5]:
print(job_list.shape)
job_list.head()

(4221, 18)


,city,company,country,date,expired,formattedLocation,formattedLocationFull,formattedRelativeTime,indeedApply,jobkey,jobtitle,onmousedown,snippet,source,sponsored,state,stations,url
0,Burbank,Kaiser Permanente,US,"Wed, 22 Feb 2017 00:08:47 GMT",False,"Burbank, CA","Burbank, CA",1 hour ago,False,fdbd1113c5011d69,Market Data Analyst - Undergraduate Internship,"indeed_clk(this,'5647');",Aptitude to perform <b>data</b> and process an...,Kaiser Permanente,False,CA,,http://www.indeed.com/viewjob?jk=fdbd1113c5011...
1,Oakland,Clorox,US,"Mon, 06 Feb 2017 23:25:16 GMT",False,"Oakland, CA","Oakland, CA 94612",15 days ago,False,456e90c7ee9c4936,Global Insights Data Analyst Data Science Inte...,"indeed_clk(this,'5647');",The Global Insights <b>Data</b> <b>Analyst</b>...,The Clorox Company,False,CA,,http://www.indeed.com/viewjob?jk=456e90c7ee9c4...
2,Redwood City,Proteus Digital Health,US,"Fri, 17 Feb 2017 04:08:45 GMT",False,"Redwood City, CA","Redwood City, CA 94065",4 days ago,False,676ec709d44d20b9,Data Analyst Intern,"indeed_clk(this,'5647');","Mine, clean, and categorize historical <b>data...",Proteus Digital Health,False,CA,,http://www.indeed.com/viewjob?jk=676ec709d44d2...
3,Santa Clara,GLOBALFOUNDRIES,US,"Wed, 08 Feb 2017 07:12:14 GMT",False,"Santa Clara, CA","Santa Clara, CA",13 days ago,False,2597a304d86609fc,Data Analyst-Intern,"indeed_clk(this,'5647');",<b>Data</b> analysis and validation. Database ...,GLOBALFOUNDRIES,False,CA,,http://www.indeed.com/viewjob?jk=2597a304d8660...
4,San Francisco,Samasource,US,"Fri, 17 Feb 2017 03:46:33 GMT",False,"San Francisco, CA","San Francisco, CA",4 days ago,False,e726041a4f3ec15b,Project/Data Analyst Internship,"indeed_clk(this,'5647');",The <b>analyst</b> intern analyzes project per...,Samasource,False,CA,,http://www.indeed.com/viewjob?jk=e726041a4f3ec...


## Problems

<ol>
    <li> Can only get at max 25 jobs per request <font color="red">Change start params</font>
    <li> When search for "Data Analyst", it is actually searching for "Data" and "Analyst" as two seperated key words
    <li> Cannot extract the job requirement information <font color="red"> Probably need web scraping </font>
    <li> 
</ol>

## Step 2. Webscraping each post

In [6]:
url_list = list(job_list['url'])

In [7]:
def get_post_details(url):
    """
    This function extracts information of the one post by taking a url as input
    Output is a dictionary with job title, company name, and description of the job.
    """
    post_doc = requests.get(url).content
    post = BeautifulSoup(post_doc, 'html.parser')
    
    title = post.find('b',attrs={'class':'jobtitle'}).text
    company = post.find(attrs={'class':'company'}).text
    dis_html = post.find(attrs={'id':'job_summary'})
    #subtitle_html = dis_html.find_all('b')
    #subtitle = [txt.text for txt in subtitle_html]
    post_dict = {'title':title, 'company':company, 'description':dis_html}
    return post_dict

In [12]:
post_summary = [get_post_details(url) for url in url_list]
post_summary = pd.DataFrame(post_summary )

In [13]:
post_summary.tail()

,company,description,title
4216,KLA-Tencor,"<span class=""summary"" id=""job_summary"">KLA-Ten...",Intern / Co-Op Technical
4217,Lyft Corporate,"<span class=""summary"" id=""job_summary"">Analyti...",Data Analyst Intern (Summer 2017)
4218,Rubin Postaer and Associates (RPA),"<span class=""summary"" id=""job_summary"">The Res...","Summer Internship - Research, Analytics + Insi..."
4219,Quidel Corporation,"<span class=""summary"" id=""job_summary"">Quidel ...","Internship, Data Analyst/Business Intelligence..."
4220,Teradata,"<span class=""summary"" id=""job_summary"">Teradat...",Data Analyst and Test Development Intern (Summ...


In [14]:
# subtitle inside job description
list(post_summary['description'])[0].find_all('b')

[<b>Essential Functions:</b>,
 <b>Department Profile:</b>,
 <b>Internship Project Summary:</b>,
 <b>Desired Skill Set:</b>,
 <b>Basic Qualifications:</b>,
 <b>Additional Requirements:</b>,
 <b>Preferred Qualifications:</b>]

In [155]:
# The whole paragraph of description
list(post_summary['description'])[0].text

"Lay the groundwork for a dynamic future with an industry leader. At Kaiser Permanente, you'll forge lasting relationships with\n\ncolleagues across the organization while enjoying the opportunity to grow your experience and hone your expertise on assignments\n\nthat have real impact. Our interns have worked on everything from analyzing financial software and developing new marketing\n\nstrategies, to designing mobile apps and helping clinicians improve care delivery. We want you to get the most out of your time here\n\nand will give you projects that challenge you to think freely, question thoroughly, and explore deeply.\n\nEssential Functions:\n\nContributing to a designated project or initiative to meet a KP business objective.\nReport development and analytics.\nProject management - planning, execution, and measurement.\nLearning new applications needed to complete assignments or support the execution of business objectives.\nJob shadowing in other functional areas.\nPerforming add

# Strategy for Extracting Qualifications
<ol>
    <li> compare all subtitles and find out which subtitles are "Qualifications"
    <li> split text by using subtitles
    <li> compare the word in qualifications and analyze
    
</ol>

In [41]:
# get subtitles inside job description
def get_subtitle(post):
    temp = post.find_all('b')
    subtitles = [sub.text for sub in temp]
    return subtitles

# a list containing the subtitles of each post
subtitles = [get_subtitle(post) for post in list(post_summary['description'])]

In [55]:
stemmer = PorterStemmer().stem

tokenize = nltk.word_tokenize

def stem(tokens,stemmer = PorterStemmer().stem):
    return [stemmer(w.lower()) for w in tokens] 

def lemmatize(text):
    """
    Extract simple lemmas based on tokenization and stemming
    Input: string
    Output: list of strings (lemmata)
    """
    return stem(tokenize(text))

stop = set(stopwords.words('english'))

def diction(files):
    """
    Returns a dictionary from lemmata to document ids containing that lemma
    Input is a data frame
    Output is a dictionary with lemmata as key and document ids as values
    """
    textd = {} 
    for i in range(len(files)):
        # loop over each raw text
        t = ' '.join(files[i])
        # return unique and order list of words appeared in the raw text
        s = set(lemmatize(t))- stop - set(string.punctuation)
        try:
            toks = toks | s   # append to "toks" set a "s"
        except NameError:
            toks = s    # if doesn't exsit, initialize it
        for tok in s:
            try:
                textd[tok].append(i)
            except KeyError:
                textd[tok] = [i]
    
    return textd

In [59]:
# Lemmata of subtitles and the id of posts containing that Lemmata
sub_id = diction(subtitles)

In [64]:
sub_num = {key:len(set(val)) for key,val in sub_id.items()}

In [71]:
# Lemmata of subtitles and the number of posts containing that Lemmata
sub_num = {key:len(set(val)) for key,val in sub_id.items()}
# Sort sub_num
sorted_sub = [(key, sub_num[key]) for key in sorted(sub_num, key=lambda x : sub_num[x],
                                                              reverse=True)]
df_sorted_sub = pd.DataFrame(sorted_sub)
df_sorted_sub.columns = ['word', '# of posts']
df_sorted_sub.iloc[:50,:]

,word,# of posts
0,skill,1206
1,requir,1005
2,function,1005
3,desir,804
4,experi,804
5,opportun,804
6,essenti,603
7,key,603
8,project,603
9,set,603


In [84]:
keywords = list(df_sorted_sub['word'][[0,1,3,11,16,17]])

def get_qual(subtitles):
    qual = [sub for sub in subtitles if set(lemmatize(sub)).intersection(keywords)]
    return qual
qual_sub = [get_qual(sub) for sub in subtitles]

In [ ]:
### use subtitles to split 